In [50]:
import pandas as pd
import numpy as np
import math
import gurobipy as gp
from gurobipy import GRB

### Preprocessing

TODO:
* Define utility functions
    * An $m$-element array, where the $j$-th element is the utility of the $j$-th class
    * Currently requires no duplicate utilities

* Define course capacities
    * An $m$-element array, where the $j$-th element is the capacity of the $j$-th class

* Define initial budgets
    * An $n$-element array
    
* Define M, number of courses
* Define N, number of students
* Define K, max. bundle size

In [51]:
# M is number of courses
# N is number of students
# K is max. number of classes
M = 4
N = 2
K = 2

In [52]:
# combinations of length n
def combinations(array, tuple_length, prev_array=[]):
    if len(prev_array) == tuple_length:
        return [prev_array]
    combs = []
    for i, val in enumerate(array):
        prev_array_extended = prev_array.copy()
        prev_array_extended.append(val)
        combs += combinations(array[i+1:], tuple_length, prev_array_extended)
    return combs

# all bundles of less than length $k$. Sorted with increasing utility
def powerset(array, max_bundle):
    subsets = []
    for i in range(max_bundle):
        subsets += combinations(array, i+1)
    return sorted(subsets, key=sum)

In [53]:
# turns a bundle not necessarily in R^m into corresponding allocation vector in R^m
def bundle_to_allocation(bundle, utility):
    m = len(utility)
    allocation = np.zeros(m)
    for i in range(len(bundle)):
        item = utility.index(bundle[i])
        allocation[item] = 1
    return allocation

# Input: utility function
#        Max bundle size
# Output: set of allocation vectors in increasing utility.
def allocations(utility, k):
    feasible_bundles = powerset(utility, k)
    allocations = []
    for i in feasible_bundles:
        allocations.append(bundle_to_allocation(i, utility))
    return allocations

# costs for an allocation vectors
def cost(allocation, prices):
    return np.dot(allocation, prices)

In [54]:
# input: set of utility-ordered allocations, utility, prices, and budget
# output: utility-maximizing budget-feasible bundles

# note: not always unique. For convenience/later implementation, maybe choose lexicographically smallest?
def demand_set(allocations, prices, utility, budget):
    demand_set = []

    # can turn this to binary search for small speedup
    for i in reversed(allocations):
        feasible = (cost(i, prices) <= budget)
        if (feasible and len(demand_set) == 0):
            demand_set.append(i)
            max_u = np.dot(i, utility)
        elif (feasible and len(demand_set) > 0):
            if(np.dot(i, utility) < max_u):
                return demand_set
            else:
                demand_set.append(i)
        
    return []


p = [0,0,0,0]

In [55]:
def budget_intervals(bundles, prices, utility, budget, delta, epsilon):
    discontinuities = []
    allocations = []
    lower = budget-epsilon
    discontinuities.append(lower)
    current_demand = demand_set(bundles, prices, utility, budget)[0]

    #find closest multiple of delta to lower
    temporary_budget = math.floor(lower/delta)

    while (temporary_budget + delta < budget+epsilon):
        new_budget = temporary_budget + delta
        new_demand = demand_set(bundles, prices, utility, new_budget)[0]
        if (set(new_demand) == set(current_demand)):
            temporary_budget += delta
            continue
        else:
            discontinuities.append(temporary_budget)
            allocations.append(new_demand)
            current_demand = new_demand
            temporary_budget += delta
    
    discontinuities.append(budget+epsilon)
    new_demand = demand_set(bundles, prices, utility, budget+epsilon)[0]
    allocations.append(new_demand)

    #Get intervals from discontinuity points
    intervals = []
    for i in range(len(discontinuities) - 1):
        intervals.append([discontinuities[i], discontinuities[i+1]])
    
    #return intervals, allocations
    return allocations


## A-CEEI

Inputs 
* Student's utility functions $u_i$
* Course capacities $c$
* Initial budgets b_0

Parameters
* Step size $\delta$
* Max. budget perturbation $\epsilon$

Outputs
* Final prices $p^* \in \mathbb{R}^m$
* Final budgets $b^* \in \mathbb{R}^n$

### Description

1) Take in $u, c, b_0$.
2) Set p = 0.
3) Find optimal budget perturbation given current prices.
4) With this set of budgets, compute clearing error.
5) Terminate if clearing error is 0. Otherwise, update prices.

In [56]:
# step size and error 
# a = allocations?
# c = course capacities

# p = prices

def clearing_error_optimizer(a, c, p):
	n=len(a)
	m=len(c)
	ki=len(a[0])
	# Create a new model
	model = gp.Model("Clearing_Error_Minimization")

	# Decision variables
	# These are the dimensions of the decision variable array
	# array of binary decision variables with n rows and ki columns
	z = model.addVars(m, lb=0.0, vtype=GRB.INTEGER, name="z")
	x = model.addVars(n, ki, vtype=GRB.BINARY, name="x")
	print(z)

	# Objective function: Minimize the l1-norm of vector z
	model.setObjective(gp.quicksum(z[j] for j in range(m)), sense=GRB.MINIMIZE)

	# Constraints
	for j in range(m):
		if p[j]>0:
			model.addConstr(gp.quicksum(x[i, l] * a[i][l][j] for i in range(n) for l in range(ki)) == c[j] + z[j], f"clearing_error_positive_{j}")
		if p[j]==0:
			model.addConstr(gp.quicksum(x[i, l] * a[i][l][j] for i in range(n) for l in range(ki)) <= c[j] + z[j], f"clearing_error_nonnegative_{j}")
	
	for i in range(n):
		model.addConstr(gp.quicksum(x[i, l] for l in range(ki)) == 1, f"one_schedule_per_student_{i}")

	# Solve the model
	model.optimize()

	# Check optimization status
	if model.status == GRB.OPTIMAL:
		# Access the optimal solution
		optimal_x = model.getAttr("x", x)
		optimal_z = model.getAttr("x", z)

		# Print or use the optimal solution as needed
		print("Optimal x values:", optimal_x)
		print("Optimal z values:", optimal_z)
		return (optimal_x, optimal_z)
	else:
		print(f"Optimization terminated with status {model.status}")
		# Check if the model is infeasible
		print(f"HELLO")
		model.computeIIS()
		print("\nInfeasible constraints:")
		for c in model.getConstrs():
			if c.IISConstr:
				print(c.constrName)
		return (None, None)

In [57]:
M = 4
N = 2
K = 2
D = 0.1
E = 0.2

budgets = np.ones(2)

# suppose we have [large diamond, small diamond, big rock, small rock]
u_1 = [10, 6, 3, 1]
u_2 = [6, 9, 1, 2]

u = [u_1, u_2]

c = [1,1,1,1]

In [58]:
def ACEEI(u, c, b_0):
    p = np.zeros(len(c))
    
    zeroClearingError = False
    
    count = 0
    for n in range(10):
        count += 1
        a = []
        for i in range(len(u)):
            x = allocations(u[i], K)
            demands = budget_intervals(x, p, u[i], b_0[i], D, E)
            a.append(demands)
        x, z = clearing_error_optimizer(a, c, p)

        error = list(z.values())

        zeroClearingError = True
        for i in error:
            if (i != 0):
                zeroClearingError = False
        
        if (zeroClearingError):
            print("\n\n\n\nExact equilibrium found after " + str(count) + " steps")
            return p, x, z
        else:
            p = np.add(p, D*np.array(list(z.values())))

    return p, x, z

In [59]:
small = ACEEI(u, c, budgets)
print("\n")
print(small[0])
print(small[1])
print(small[2])

{0: <gurobi.Var *Awaiting Model Update*>, 1: <gurobi.Var *Awaiting Model Update*>, 2: <gurobi.Var *Awaiting Model Update*>, 3: <gurobi.Var *Awaiting Model Update*>}
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.1.0 23B92)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 6 rows, 6 columns and 10 nonzeros
Model fingerprint: 0x6e667f30
Variable types: 0 continuous, 6 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e